# Run QARTOD Test on Locally Saved Data

In this notebook we will load locally saved data from the interim data folder, extract QARTOD test parameters from spreadsheets on the OOI GitHub, run the QARTOD climatology and gross range tests on the imported data, and save the test results to the processed data folder.

More info about QARTOD tests and the ioos_qc module can be found from the [Integrated Ocean Observing System website](https://ioos.noaa.gov/project/qartod/) and [Python module documentation](https://ioos.github.io/ioos_qc/), respectively.

### Import modules used in this notebook

In [1]:
# Import libraries
import os
import re
import gc
import io
import ast
import pandas as pd
import numpy as np
import xarray as xr
import warnings
warnings.filterwarnings("ignore")
import sys

# Import IOOS module with QARTOD tests
import ioos_qc

# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

In [2]:
# Import OOINet library - might delete
sys.path.append("c:\\Users\\cooleyky\\Documents\\GitHub\\OOINet") # this is what was missing from the steps I followed to install ooinet and ooi-data-explorations as local dev repo
from ooinet import M2M
from ooinet.Instrument.common import process_file

In [3]:
# Import functions from ooi-data-explorations library - might delete
sys.path.append("c:\\Users\\cooleyky\\Documents\\GitHub\\ooi-data-explorations\\python") # why did the initial install not include this?
from ooi_data_explorations.uncabled.process_dosta import dosta_datalogger
from ooi_data_explorations.combine_data import combine_datasets

### Load locally saved data

In [4]:
# Use reference designator, data stream, and method to build file name and path for chosen dataset

site = "CP01CNSM"                                   # Coastal Pioneer Array (NES) - Central Surface Mooring
node = "MFD37"                                      # 
sensor = "03-CTDBPD000"                             # CTD Bottom-pumped
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

refdes = '-'.join((site,node,sensor))               # build reference designator

type = 'prod'                                       # dataset saved from OOINet/"production" or from dev1

In [10]:
# Build filename and path to interim data

def test_data_path(refdes,method,stream,type,folder='interim'):
    # Input: 
    #   refdes: string built from OOI site, node, and sensor for chosen dataset
    #   method: 'recovered_inst', 'recovered_host', or 'telemetered'(?) 
    #   stream: name of data stream 
    #   type: 'prod' or 'dev'
    #   folder: 'interim' (default), 'processed', 'raw', or 'external'
    #
    # Returns:
    #   ds_path: relative path to dataset from notebook folder
    
    filename = '-'.join((type,refdes,method,stream))+'.nc'              # build filename from dataset type and source

    data_folder = os.path.relpath('../data')                            # path to data folder from notebook folder

    ds_path=os.path.join(data_folder,folder,filename)                   # build full relative path 
    
    return ds_path

ds_path = test_data_path(refdes,method,stream,'prod')
ds_path

'..\\data\\interim\\prod-CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered.nc'

In [11]:
# Load data from .nc files

ds = xr.open_dataset(ds_path)
ds

<xarray.Dataset>
Dimensions:                                            (time: 263601)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) uint8 ...
    sea_water_pressure                                 (time) float64 ...
    sea_water_electrical_conductivity_qartod_results   (time) uint8 ...
    sea_water_pressure_qc_executed                     (time) uint8 ...
    ctd_time                                           (time) datetime64[ns] ...
    sea_water_practical_salinity_qc_executed           (time) uint8 ...
    ...                                                 ...
    sea_water_density                                  (time) float64 ...
    depth                                              (time) float64 ...
    sea_water_pressure_qartod_results                  (time) uint8 ...
    sea_water_electrical_conductivity_qartod_executed  (time) object ...
    sea_water_temperature                              (time) float64 ...
    sea_water_practical_salinity_qartod_executed       (time) object ...
Attributes: (12/69)
    node:                               MFD37
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13678
    lon:                                -70.76978

### Identify Test Parameters

Next, identify which parameters in the dataset have QARTOD applied to them. Sometimes the variable name in the dataset is different that the key that is used by OOINet to build the datasets. For that we can check the attributes of the variable for the "alternate_parameter_name"!

In [8]:
# Create a dictionary of key-value pairs of dataset variable name:alternate parameter name
test_parameters={}
for var in ds_prod.variables:
    if "qartod_results" in var:
        # Get the parameter name
        param = var.split("_qartod")[0]
        
        # Check if the parameter has an alternative ooinet_name
        if "alternate_parameter_name" in ds_prod[param].attrs:
            ooinet_name = ds_prod[param].attrs["alternate_parameter_name"]
        else:
            ooinet_name = param
        
        # Save the results in a dictionary
        test_parameters.update({
            param: ooinet_name
        })
# Print out the results
test_parameters

{'sea_water_electrical_conductivity': 'ctdbp_seawater_conductivity',
 'sea_water_temperature': 'ctdbp_seawater_temperature',
 'sea_water_practical_salinity': 'practical_salinity',
 'sea_water_pressure': 'ctdbp_seawater_pressure'}

### Collect test QARTOD value tables from GitHub

### Run QARTOD tests locally

### Save test results to processed data folder

In [ ]:
results_path = test_data_path(refdes,method,stream,(type+'-result'),folder='processed') # Build path with filename to folder for saved results

local_qartod_result.to_netcdf(results_path)                                             # write netCDF file with results to processed data folder